In [1]:
import numpy as np # Library for vectorized data
import pandas as pd # Library for data analysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
import json # Library for JSON files
from geopy.geocoders import Nominatim # Converting an address into Latitude/Longitude values
import requests # Library for requests of data
from bs4 import BeautifulSoup # library to parse out HTML and XML documents
from pandas.io.json import json_normalize # Tranforming a JSON file into a pandas dataframe
import matplotlib.cm as cm # Matplotlib and associated plotting modules
import matplotlib.colors as colors
from sklearn.cluster import KMeans # importing k-means from clustering stage
!pip install folium
import folium # Library for map-rendering
print("Libraries imported.")

     |████████████████████████████████| 102kB 6.8MB/s ta 0:00:011
Libraries imported.


In [2]:
TorontoData = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(TorontoData, 'html.parser')
PostalCodeList = []
BoroughList = []
NeighborhoodList = []

In [3]:
# Sorting data into their respective lists
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        PostalCodeList.append(cells[0].text)
        BoroughList.append(cells[1].text)
        NeighborhoodList.append(cells[2].text.rstrip('\n')) # Avoiding any new lines in neighborhood cell

In [4]:
# New Dataframe
Toronto_df = pd.DataFrame({"PostalCode": PostalCodeList, "Borough": BoroughList, "Neighborhood": NeighborhoodList})
Toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned
1,M2A\n,Not assigned\n,Not assigned
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"


In [5]:
# Get rid of 'Not assigned' boroughs
Toronto_df_DropNA = Toronto_df[Toronto_df.Borough != "Not assigned\n"].reset_index(drop=True)
Toronto_df_DropNA.head()

,PostalCode,Borough,Neighborhood
0,M3A\n,North York\n,Parkwoods
1,M4A\n,North York\n,Victoria Village
2,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"
3,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights"
4,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government"


In [6]:
# Grouping neighborhoods that are in the same borough
Toronto_df_Grouped = Toronto_df_DropNA.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
Toronto_df_Grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B\n,Scarborough\n,"Malvern, Rouge"
1,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek"
2,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill"
3,M1G\n,Scarborough\n,Woburn
4,M1H\n,Scarborough\n,Cedarbrae


In [7]:
# For Neighborhoods that are "Not assigned", make the value the same as Borough
for index, row in Toronto_df_Grouped.iterrows():
    if row["Neighborhood"] == "Not assigned\n":
        row["Neighborhood"] = row["Borough"]
Toronto_df_Grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B\n,Scarborough\n,"Malvern, Rouge"
1,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek"
2,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill"
3,M1G\n,Scarborough\n,Woburn
4,M1H\n,Scarborough\n,Cedarbrae


In [8]:
# Test dataframe, just to check it is all done to the requirements of the assessment.
column_names = ["PostalCode", "Borough", "Neighborhood"]
Test_df = pd.DataFrame(columns=column_names)
Test_list = ["M5G\n", "M2H\n", "M4B\n", "M1J\n", "M4G\n", "M4M\n", "M1R\n", "M9V\n", "M9L\n", "M5V\n", "M1B\n", "M5A\n"]
for postcode in Test_list:
    Test_df = Test_df.append(Toronto_df_Grouped[Toronto_df_Grouped["PostalCode"]==postcode], ignore_index=True)
Test_df

,PostalCode,Borough,Neighborhood
0,M5G\n,Downtown Toronto\n,Central Bay Street
1,M2H\n,North York\n,Hillcrest Village
2,M4B\n,East York\n,"Parkview Hill, Woodbine Gardens"
3,M1J\n,Scarborough\n,Scarborough Village
4,M4G\n,East York\n,Leaside
5,M4M\n,East Toronto\n,Studio District
6,M1R\n,Scarborough\n,"Wexford, Maryvale"
7,M9V\n,Etobicoke\n,"South Steeles, Silverstone, Humbergate, Jamest..."
8,M9L\n,North York\n,Humber Summit
9,M5V\n,Downtown Toronto\n,"CN Tower, King and Spadina, Railway Lands, Har..."


In [9]:
# Number of rows of the cleaned dataframe
Toronto_df_Grouped.shape

(103, 3)

In [10]:
#Removing useless characters from 'PostalCode' & 'Borough' columns.
Toronto_df_Grouped['PostalCode']=Toronto_df_Grouped['PostalCode'].str.replace('\n','')
Toronto_df_Grouped['Borough']=Toronto_df_Grouped['Borough'].str.replace('\n','')
Toronto_df_Grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [11]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
# Renaming the column "PostalCode"
df_data_1.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
df_data_1.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
# Merging two tables on the column "PostalCode"
Toronto_df_new = Toronto_df_Grouped.merge(df_data_1, on="PostalCode", how="left")
Toronto_df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [15]:
# Creating a new test dataframe
column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
test_df_2 = pd.DataFrame(columns=column_names)
test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]
for postcode in test_list:
    test_df_2 = test_df_2.append(Toronto_df_new[Toronto_df_new["PostalCode"]==postcode], ignore_index=True)
test_df_2

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420


In [16]:
#Attaining the latitude and longitude values of Toronto
address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [18]:
#Creating a map of Toronto using the latitude/longitude values, with extra markers on the map for the given neighborhoods.

Toronto_Map = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, neighborhood in zip(Toronto_df_new['Latitude'], Toronto_df_new['Longitude'], Toronto_df_new['Borough'], Toronto_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='black',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5).add_to(Toronto_Map)

Toronto_Map

In [21]:
# Filtering for borough names that contain the word Toronto
borough_names = list(Toronto_df_new.Borough.unique())
borough_with_Toronto = []
for x in borough_names:
    if "toronto" in x.lower():
        borough_with_Toronto.append(x)
        
borough_with_Toronto

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

In [22]:
# Creating a new DataFrame with only boroughs that have the word Toronto
TorontoB_df_new = Toronto_df_new[Toronto_df_new['Borough'].isin(borough_with_Toronto)].reset_index(drop=True)
print(TorontoB_df_new.shape)
TorontoB_df_new.head()

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [25]:
# Creating a map of Toronto using latitude/longitude values and new markers for the designated boroughs.
Toronto_Map_2 = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, neighborhood in zip(TorontoB_df_new['Latitude'], TorontoB_df_new['Longitude'], TorontoB_df_new['Borough'], TorontoB_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='black',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5).add_to(Toronto_Map_2)  
    
Toronto_Map_2

In [26]:
# Using FourSquare API to explore the given Toronto boroughs
CLIENT_ID = 'WAOOPU25ROTMGXYZM4KTFRFVMQWR0XPPPS51NSFCMRJ00YEL' # Foursquare ID
CLIENT_SECRET = '5MGHA3GTKY4BLBMXOROVATFM41HITVB13AQWGSNUK4PKYAZ3' # Foursquare Secret
VERSION = '20200715' # Foursquare API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: WAOOPU25ROTMGXYZM4KTFRFVMQWR0XPPPS51NSFCMRJ00YEL
CLIENT_SECRET:5MGHA3GTKY4BLBMXOROVATFM41HITVB13AQWGSNUK4PKYAZ3


In [28]:
#Finding top 100 venues within a 500m radius
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(TorontoB_df_new['Latitude'], TorontoB_df_new['Longitude'], TorontoB_df_new['PostalCode'], TorontoB_df_new['Borough'], TorontoB_df_new['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [30]:
#Converting the Venues list into a new DataFrame with new column names
Venues_df = pd.DataFrame(venues)

Venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(Venues_df.shape)
Venues_df.head()

(1620, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


In [32]:
#Checking how many values were returned for each Postal Code, the number of unique categories for all of them and seeing what some of those categories are.
Venues_df.groupby(["PostalCode", "Borough", "Neighborhood"]).count()
print('There are {} unique categories.'.format(len(Venues_df['VenueCategory'].unique())))
Venues_df['VenueCategory'].unique()[:50]

There are 226 unique categories.


array(['Trail', 'Health Food Store', 'Pub', 'Neighborhood',
       'Cosmetics Shop', 'Greek Restaurant', 'Ice Cream Shop',
       'Italian Restaurant', 'Brewery', 'Juice Bar', 'Yoga Studio',
       'Restaurant', 'Fruit & Vegetable Store', 'Pizza Place',
       'Bookstore', 'Dessert Shop', 'Furniture / Home Store',
       'Bubble Tea Shop', 'Spa', 'Grocery Store', 'Café', 'Bakery',
       'Coffee Shop', 'Caribbean Restaurant', 'Indian Restaurant',
       'Lounge', 'American Restaurant', 'Frozen Yogurt Shop',
       'Sushi Restaurant', 'Liquor Store', 'Gym', 'Fast Food Restaurant',
       'Fish & Chips Shop', 'Park', 'Burrito Place', 'Pet Store',
       'Steakhouse', 'Movie Theater', 'Sandwich Place',
       'Light Rail Station', 'Fish Market', 'Gay Bar', 'Cheese Shop',
       'Seafood Restaurant', 'Middle Eastern Restaurant',
       'Stationery Store', 'Comfort Food Restaurant', 'Thai Restaurant',
       'Coworking Space', 'Wine Bar'], dtype=object)

In [33]:
#Analyzing each area
# Using the one hot encoding method
Toronto_OneHot = pd.get_dummies(Venues_df[['VenueCategory']], prefix="", prefix_sep="")
Toronto_OneHot['PostalCode'] = Venues_df['PostalCode'] 
Toronto_OneHot['Borough'] = Venues_df['Borough'] 
Toronto_OneHot['Neighborhoods'] = Venues_df['Neighborhood'] 
fixed_columns = list(Toronto_OneHot.columns[-3:]) + list(Toronto_OneHot.columns[:-3])
Toronto_OneHot = Toronto_OneHot[fixed_columns]

print(Toronto_OneHot.shape)
Toronto_OneHot.head()

(1620, 229)


,PostalCode,Borough,Neighborhoods,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Cable Car,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hospital,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [36]:
#Grouping rows by neighborhood, via the mean of frequency of ech category occurring.
Toronto_grouped = Toronto_OneHot.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()
print(Toronto_grouped.shape)
Toronto_grouped

#Creating a new DataFrame, with the top 20 venues in each Postal Code.
num_top_venues = 20

indicators = ['st', 'nd', 'rd']

areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = Toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = Toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = Toronto_grouped['Neighborhoods']

for ind in np.arange(Toronto_grouped.shape[0]):
    row_categories = Toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(39, 229)
(39, 23)


,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,M4E,East Toronto,The Beaches,Pub,Neighborhood,Health Food Store,Trail,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Doner Restaurant,Department Store,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Dessert Shop,Dance Studio,Deli / Bodega,Farmers Market
1,M4K,East Toronto,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Restaurant,Ice Cream Shop,Yoga Studio,Spa,Juice Bar,Brewery,Bubble Tea Shop,Liquor Store,Lounge,Café,Caribbean Restaurant,Pub,Pizza Place,Dessert Shop,Cosmetics Shop,Indian Restaurant
2,M4L,East Toronto,"India Bazaar, The Beaches West",Park,Sandwich Place,Sushi Restaurant,Brewery,Burrito Place,Restaurant,Pub,Pizza Place,Pet Store,Coffee Shop,Movie Theater,Liquor Store,Light Rail Station,Fast Food Restaurant,Italian Restaurant,Fish & Chips Shop,Ice Cream Shop,Steakhouse,Gym,Bakery
3,M4M,East Toronto,Studio District,Café,Coffee Shop,Gastropub,American Restaurant,Bakery,Brewery,Yoga Studio,Pet Store,Park,Neighborhood,Middle Eastern Restaurant,Latin American Restaurant,Italian Restaurant,Ice Cream Shop,Gym / Fitness Center,Gay Bar,Coworking Space,Fish Market,Diner,Seafood Restaurant
4,M4N,Central Toronto,Lawrence Park,Park,Bus Line,Swim School,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Department Store,Farmers Market,Deli / Bodega,Dance Studio,Cupcake Shop,Cuban Restaurant
5,M4P,Central Toronto,Davisville North,Hotel,Department Store,Park,Food & Drink Shop,Sandwich Place,Breakfast Spot,Gym / Fitness Center,Convention Center,Concert Hall,Comic Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Cosmetics Shop
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",Clothing Store,Coffee Shop,Chinese Restaurant,Shoe Store,Seafood Restaurant,Salon / Barbershop,Restaurant,Rental Car Location,Café,Yoga Studio,Sporting Goods Shop,Miscellaneous Shop,Mexican Restaurant,Diner,Fast Food Restaurant,Gym / Fitness Center,Spa,Park,Bakery,Dessert Shop
7,M4S,Central Toronto,Davisville,Sandwich Place,Dessert Shop,Gym,Coffee Shop,Italian Restaurant,Café,Pizza Place,Sushi Restaurant,Toy / Game Store,Greek Restaurant,Brewery,Discount Store,Farmers Market,Diner,Restaurant,Japanese Restaurant,Gourmet Shop,Indian Restaurant,Pharmacy,Gas Station
8,M4T,Central Toronto,"Moore Park, Summerhill East",Lawyer,Trail,Tennis Court,Yoga Studio,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop,Deli / Bodega,Farmers Market,Dance Studio,Cupcake Shop
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",Pub,Coffee Shop,Light Rail Station,Bank,Sushi Restaurant,Restaurant,Fried Chicken Joint,Sports Bar,Pizza Place,American Restaurant,Liquor Store,Vietnamese Restaurant,Supermarket,Dog Run,Doner Restaurant,Distribution Center,Discount Store,Diner,Donut Shop,Dessert Shop


In [41]:
#Using the k-means method for separating the area into 5 clusters.
kclusters = 5
Toronto_grouped_clustering = Toronto_grouped.drop(["PostalCode", "Borough", "Neighborhoods"], 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)
kmeans.labels_[0:20]

#Creating a new dataframe that includes the clusters as well as the top 20 venues for each neighborhood.
Toronto_Final = TorontoB_df_new.copy()
Toronto_Final["Cluster Labels"] = kmeans.labels_
Toronto_Final = Toronto_Final.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("PostalCode"), on="PostalCode")
print(Toronto_Final.shape)
Toronto_Final.head()

(39, 26)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Pub,Neighborhood,Health Food Store,Trail,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Doner Restaurant,Department Store,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Dessert Shop,Dance Studio,Deli / Bodega,Farmers Market
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Restaurant,Ice Cream Shop,Yoga Studio,Spa,Juice Bar,Brewery,Bubble Tea Shop,Liquor Store,Lounge,Café,Caribbean Restaurant,Pub,Pizza Place,Dessert Shop,Cosmetics Shop,Indian Restaurant
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Park,Sandwich Place,Sushi Restaurant,Brewery,Burrito Place,Restaurant,Pub,Pizza Place,Pet Store,Coffee Shop,Movie Theater,Liquor Store,Light Rail Station,Fast Food Restaurant,Italian Restaurant,Fish & Chips Shop,Ice Cream Shop,Steakhouse,Gym,Bakery
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Gastropub,American Restaurant,Bakery,Brewery,Yoga Studio,Pet Store,Park,Neighborhood,Middle Eastern Restaurant,Latin American Restaurant,Italian Restaurant,Ice Cream Shop,Gym / Fitness Center,Gay Bar,Coworking Space,Fish Market,Diner,Seafood Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Park,Bus Line,Swim School,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Department Store,Farmers Market,Deli / Bodega,Dance Studio,Cupcake Shop,Cuban Restaurant


In [43]:
#Sorting the DataFrane by Cluster Labels
print(Toronto_Final.shape)
Toronto_Final.sort_values(["Cluster Labels"], inplace=True)
Toronto_Final

(39, 26)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Pub,Neighborhood,Health Food Store,Trail,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Doner Restaurant,Department Store,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Dessert Shop,Dance Studio,Deli / Bodega,Farmers Market
20,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576,0,Coffee Shop,Hotel,Café,Salad Place,Japanese Restaurant,Italian Restaurant,Restaurant,American Restaurant,Seafood Restaurant,Sushi Restaurant,Deli / Bodega,Beer Bar,Sporting Goods Shop,Steakhouse,Concert Hall,Tea Room,Pub,Train Station,Gym / Fitness Center,Roof Deck
21,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,0,Coffee Shop,Restaurant,Café,Hotel,Gym,American Restaurant,Deli / Bodega,Seafood Restaurant,Italian Restaurant,Japanese Restaurant,Cocktail Bar,Tea Room,Beer Bar,Gastropub,Vegetarian / Vegan Restaurant,Thai Restaurant,New American Restaurant,Building,Creperie,Monument / Landmark
24,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,0,Sandwich Place,Café,Coffee Shop,Park,History Museum,Shoe Repair,Burger Joint,Middle Eastern Restaurant,Indian Restaurant,Pub,Liquor Store,BBQ Joint,Donut Shop,Pizza Place,Vegetarian / Vegan Restaurant,Pharmacy,Pet Store,Grocery Store,Comic Shop,Concert Hall
25,M5S,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049,0,Café,Bar,Japanese Restaurant,Bookstore,Sandwich Place,Restaurant,Bakery,Yoga Studio,Pub,Beer Bar,Beer Store,Coffee Shop,Chinese Restaurant,College Gym,Comfort Food Restaurant,Noodle House,Nightclub,Dessert Shop,Italian Restaurant,French Restaurant
26,M5T,Downtown Toronto,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049,0,Café,Coffee Shop,Vietnamese Restaurant,Bar,Mexican Restaurant,Bakery,Dessert Shop,Vegetarian / Vegan Restaurant,Gaming Cafe,Park,Grocery Store,Pizza Place,Farmers Market,Organic Grocery,Breakfast Spot,Belgian Restaurant,Smoothie Shop,Filipino Restaurant,Burger Joint,Japanese Restaurant
27,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,0,Airport Service,Airport Terminal,Airport,Harbor / Marina,Rental Car Location,Coffee Shop,Sculpture Garden,Boat or Ferry,Airport Lounge,Airport Food Court,Bar,Event Space,Ethiopian Restaurant,Discount Store,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run
28,M5W,Downtown Toronto,Stn A PO Boxes,43.646435,-79.374846,0,Coffee Shop,Café,Cocktail Bar,Restaurant,Japanese Restaurant,Seafood Restaurant,Italian Restaurant,Hotel,Beer Bar,Gym,Pharmacy,Lounge,Farmers Market,Creperie,Pub,Cheese Shop,Art Gallery,Breakfast Spot,Bakery,Indian Restaurant
29,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280,0,Coffee Shop,Café,Hotel,Gym,Japanese Restaurant,Restaurant,Asian Restaurant,Deli / Bodega,American Restaurant,Seafood Restaurant,Steakhouse,Pizza Place,Salad Place,Beer Bar,Sushi Restaurant,Concert Hall,Thai Restaurant,Tea Room,Gastropub,Building
30,M6G,Downtown Toronto,Christie,43.669542,-79.422564,0,Grocery Store,Café,Park,Athletics & Sports,Italian Restaurant,Candy Store,Diner,Restaurant,Baby Store,Coffee Shop,Nightclub,Creperie,Discount Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


In [46]:
#Creating map of the clusters
ClusterMap = folium.Map(location=[latitude, longitude], zoom_start=10)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

Markers_Colors = []
for lat, lon, post, bor, poi, cluster in zip(Toronto_Final['Latitude'], Toronto_Final['Longitude'], Toronto_Final['PostalCode'], Toronto_Final['Borough'], Toronto_Final['Neighborhood'], Toronto_Final['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=4,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.5).add_to(ClusterMap)
ClusterMap

In [48]:
#Analyzing each Cluster

#Cluster 1
Toronto_Final.loc[Toronto_Final['Cluster Labels'] == 0, Toronto_Final.columns[[1] + list(range(5, Toronto_Final.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,East Toronto,0,Pub,Neighborhood,Health Food Store,Trail,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Doner Restaurant,Department Store,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Dessert Shop,Dance Studio,Deli / Bodega,Farmers Market
20,Downtown Toronto,0,Coffee Shop,Hotel,Café,Salad Place,Japanese Restaurant,Italian Restaurant,Restaurant,American Restaurant,Seafood Restaurant,Sushi Restaurant,Deli / Bodega,Beer Bar,Sporting Goods Shop,Steakhouse,Concert Hall,Tea Room,Pub,Train Station,Gym / Fitness Center,Roof Deck
21,Downtown Toronto,0,Coffee Shop,Restaurant,Café,Hotel,Gym,American Restaurant,Deli / Bodega,Seafood Restaurant,Italian Restaurant,Japanese Restaurant,Cocktail Bar,Tea Room,Beer Bar,Gastropub,Vegetarian / Vegan Restaurant,Thai Restaurant,New American Restaurant,Building,Creperie,Monument / Landmark
24,Central Toronto,0,Sandwich Place,Café,Coffee Shop,Park,History Museum,Shoe Repair,Burger Joint,Middle Eastern Restaurant,Indian Restaurant,Pub,Liquor Store,BBQ Joint,Donut Shop,Pizza Place,Vegetarian / Vegan Restaurant,Pharmacy,Pet Store,Grocery Store,Comic Shop,Concert Hall
25,Downtown Toronto,0,Café,Bar,Japanese Restaurant,Bookstore,Sandwich Place,Restaurant,Bakery,Yoga Studio,Pub,Beer Bar,Beer Store,Coffee Shop,Chinese Restaurant,College Gym,Comfort Food Restaurant,Noodle House,Nightclub,Dessert Shop,Italian Restaurant,French Restaurant
26,Downtown Toronto,0,Café,Coffee Shop,Vietnamese Restaurant,Bar,Mexican Restaurant,Bakery,Dessert Shop,Vegetarian / Vegan Restaurant,Gaming Cafe,Park,Grocery Store,Pizza Place,Farmers Market,Organic Grocery,Breakfast Spot,Belgian Restaurant,Smoothie Shop,Filipino Restaurant,Burger Joint,Japanese Restaurant
27,Downtown Toronto,0,Airport Service,Airport Terminal,Airport,Harbor / Marina,Rental Car Location,Coffee Shop,Sculpture Garden,Boat or Ferry,Airport Lounge,Airport Food Court,Bar,Event Space,Ethiopian Restaurant,Discount Store,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run
28,Downtown Toronto,0,Coffee Shop,Café,Cocktail Bar,Restaurant,Japanese Restaurant,Seafood Restaurant,Italian Restaurant,Hotel,Beer Bar,Gym,Pharmacy,Lounge,Farmers Market,Creperie,Pub,Cheese Shop,Art Gallery,Breakfast Spot,Bakery,Indian Restaurant
29,Downtown Toronto,0,Coffee Shop,Café,Hotel,Gym,Japanese Restaurant,Restaurant,Asian Restaurant,Deli / Bodega,American Restaurant,Seafood Restaurant,Steakhouse,Pizza Place,Salad Place,Beer Bar,Sushi Restaurant,Concert Hall,Thai Restaurant,Tea Room,Gastropub,Building
30,Downtown Toronto,0,Grocery Store,Café,Park,Athletics & Sports,Italian Restaurant,Candy Store,Diner,Restaurant,Baby Store,Coffee Shop,Nightclub,Creperie,Discount Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


In [49]:
#Cluster 2
Toronto_Final.loc[Toronto_Final['Cluster Labels'] == 1, Toronto_Final.columns[[1] + list(range(5, Toronto_Final.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
22,Central Toronto,1,Home Service,Garden,Yoga Studio,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Deli / Bodega,Department Store,Fast Food Restaurant,Dance Studio,Cupcake Shop


In [50]:
#Cluster 3
Toronto_Final.loc[Toronto_Final['Cluster Labels'] == 2, Toronto_Final.columns[[1] + list(range(5, Toronto_Final.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
8,Central Toronto,2,Lawyer,Trail,Tennis Court,Yoga Studio,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop,Deli / Bodega,Farmers Market,Dance Studio,Cupcake Shop


In [51]:
#Cluster 4
Toronto_Final.loc[Toronto_Final['Cluster Labels'] == 3, Toronto_Final.columns[[1] + list(range(5, Toronto_Final.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
4,Central Toronto,3,Park,Bus Line,Swim School,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Department Store,Farmers Market,Deli / Bodega,Dance Studio,Cupcake Shop,Cuban Restaurant
10,Downtown Toronto,3,Park,Playground,Trail,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store,Cupcake Shop,Dance Studio,Falafel Restaurant,Cuban Restaurant,Creperie


In [52]:
#Cluster 5
Toronto_Final.loc[Toronto_Final['Cluster Labels'] == 4, Toronto_Final.columns[[1] + list(range(5, Toronto_Final.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
23,Central Toronto,4,Park,Trail,Jewelry Store,Bus Line,Sushi Restaurant,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Department Store,Falafel Restaurant,Deli / Bodega,Dance Studio


Conclusion

From looking at the given clusters, it would seem each cluster have generally different focuses, at least in regards to the venues they hold. In Cluster 1, the neighborhoods tend to fall into business areas, with plenty of eating areas, like Cafes, Cffee Shops, Restaurants etc. In Cluster 2, it is just a garden. Cluster 3 is a Lawyer firm, with a nearby Trail. Cluster 4 is a park with a nearby playground, swim school and a few eating areas. Finally, Cluster 5 is a park and trail with a couple of eating areas nearby.